In [1]:
import warnings
warnings.filterwarnings("ignore")

from xai_agg.agg_exp import *
from xai_agg.utils import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import pandas as pd

2025-01-11 13:14:37.119785: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-11 13:14:37.143210: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocess the data
1. One-hot-encode categorical variables, making sure the one-hot-encoded column names are in the format "[FEATURE]_[CATEGORY]"
2. Make sure all column names are valid python identifiers

In [2]:
original_data = pd.read_csv('../data/german_credit_data_updated.csv')

# Dataset overview - German Credit Risk (from Kaggle):
# 1. Age (numeric)
# 2. Sex (text: male, female)
# 3. Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
# 4. Housing (text: own, rent, or free)
# 5. Saving accounts (text - little, moderate, quite rich, rich)
# 6. Checking account (numeric, in DM - Deutsch Mark)
# 7. Credit amount (numeric, in DM)
# 8. Duration (numeric, in month)
# 9. Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

display(original_data.head())
display(original_data.describe())
display(original_data.info())

# Display the unique values of the categorical features:
print('Unique values of the categorical features:')
for col in original_data.select_dtypes(include='object'):
    print(f'\t- {col}: {original_data[col].unique()}')

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Credit Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,1
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,2,49,male,1,own,little,NaN,2096,12,education,1
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,4,53,male,2,free,little,little,4870,24,car,2


,Unnamed: 0,Age,Job,Credit amount,Duration,Credit Risk
count,954.000000,954.000000,954.000000,954.000000,954.000000,954.000000
mean,476.500000,35.501048,1.909853,3279.112159,20.780922,1.302935
std,275.540378,11.379668,0.649681,2853.315158,12.046483,0.459768
min,0.000000,19.000000,0.000000,250.000000,4.000000,1.000000
25%,238.250000,27.000000,2.000000,1360.250000,12.000000,1.000000
50%,476.500000,33.000000,2.000000,2302.500000,18.000000,1.000000
75%,714.750000,42.000000,2.000000,3975.250000,24.000000,2.000000
max,953.000000,75.000000,3.000000,18424.000000,72.000000,2.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        954 non-null    int64 
 1   Age               954 non-null    int64 
 2   Sex               954 non-null    object
 3   Job               954 non-null    int64 
 4   Housing           954 non-null    object
 5   Saving accounts   779 non-null    object
 6   Checking account  576 non-null    object
 7   Credit amount     954 non-null    int64 
 8   Duration          954 non-null    int64 
 9   Purpose           954 non-null    object
 10  Credit Risk       954 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 82.1+ KB


None

Unique values of the categorical features:
	- Sex: ['male' 'female']
	- Housing: ['own' 'free' 'rent']
	- Saving accounts: [nan 'little' 'quite rich' 'rich' 'moderate']
	- Checking account: ['little' 'moderate' nan 'rich']
	- Purpose: ['radio/TV' 'education' 'furniture/equipment' 'car' 'business'
 'domestic appliances' 'repairs' 'vacation/others']


In [3]:
preprocessed_data = original_data.copy()

# For savings and checking accounts, we will replace the missing values with 'none':
preprocessed_data['Saving accounts'].fillna('none', inplace=True)
preprocessed_data['Checking account'].fillna('none', inplace=True)

# Dropping index column:
preprocessed_data.drop(columns=['Unnamed: 0'], inplace=True)

# Using pd.dummies to one-hot-encode the categorical features
preprocessed_data["Job"] = preprocessed_data["Job"].map({0: 'unskilled_nonresident', 1: 'unskilled_resident',
                                                         2: 'skilled', 3: 'highlyskilled'})

categorical_features = preprocessed_data.select_dtypes(include='object').columns
numerical_features = preprocessed_data.select_dtypes(include='number').columns.drop('Credit Risk')
print(f'Categorical features: {categorical_features}')
print(f'Numerical features: {numerical_features}')

preprocessed_data = pd.get_dummies(preprocessed_data, columns=categorical_features, dtype='int64')

# Remapping the target variable to 0 and 1:
preprocessed_data['Credit Risk'] = preprocessed_data['Credit Risk'].map({1: 0, 2: 1})

# Make sure all column names are valid python identifiers (important for pd.query() calls):
preprocessed_data.columns = preprocessed_data.columns.str.replace(' ', '_')
preprocessed_data.columns = preprocessed_data.columns.str.replace('/', '_')

# Normalizing the data
scaler = StandardScaler()
scaled_preprocessed_data = scaler.fit_transform(preprocessed_data)

display(preprocessed_data.head())
display(preprocessed_data.info())

display(scaled_preprocessed_data)

Categorical features: Index(['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Purpose'],
      dtype='object')
Numerical features: Index(['Age', 'Credit amount', 'Duration'], dtype='object')


,Age,Credit_amount,Duration,Credit_Risk,Sex_female,Sex_male,Job_highlyskilled,Job_skilled,Job_unskilled_nonresident,Job_unskilled_resident,...,Checking_account_none,Checking_account_rich,Purpose_business,Purpose_car,Purpose_domestic_appliances,Purpose_education,Purpose_furniture_equipment,Purpose_radio_TV,Purpose_repairs,Purpose_vacation_others
0,67,1169,6,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,22,5951,48,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,49,2096,12,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
3,45,7882,42,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,53,4870,24,1,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age                          954 non-null    int64
 1   Credit_amount                954 non-null    int64
 2   Duration                     954 non-null    int64
 3   Credit_Risk                  954 non-null    int64
 4   Sex_female                   954 non-null    int64
 5   Sex_male                     954 non-null    int64
 6   Job_highlyskilled            954 non-null    int64
 7   Job_skilled                  954 non-null    int64
 8   Job_unskilled_nonresident    954 non-null    int64
 9   Job_unskilled_resident       954 non-null    int64
 10  Housing_free                 954 non-null    int64
 11  Housing_own                  954 non-null    int64
 12  Housing_rent                 954 non-null    int64
 13  Saving_accounts_little       954 non-null    int64

None

array([[ 2.7694545 , -0.7399179 , -1.22763429, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [-1.18704073,  0.93690642,  2.26068929, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [ 1.18685641, -0.41486224, -0.72930235, ..., -0.61531514,
        -0.14633276, -0.11286653],
       ...,
       [-1.0111965 , -0.39768023,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.65950803,  0.29240557,  0.26736153, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.83535227,  2.69823821,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653]])

In [4]:
y = preprocessed_data['Credit_Risk']
X = preprocessed_data.drop(columns='Credit_Risk')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')

Accuracy: 0.7696335078534031
ROC AUC: 0.6830357142857143


# Applying the Aggregate Explainer

In [6]:
agg_explainer = AggregatedExplainer(
    explainer_types=[LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper],       # Wrapped explainers whose explanations will be aggregated
    clf=clf, X_train=X_train, categorical_feature_names=categorical_features,   # Model and training data
    metrics=['nrc', 'sensitivity_spearman', 'faithfulness_corr'],               # Metrics to be considered for the aggregation
    noise_gen_args={'encoding_dim': 5, 'epochs': 500},                          # Arguments passed to the autoencoder noisy data generator
    evaluator_args={"debug": False}                                             # Arguments passed to the evaluator class 
)                                                       

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2655 - val_loss: 1.2404
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2126 - val_loss: 1.2241
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2003 - val_loss: 1.2084
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2105 - val_loss: 1.1930
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1804 - val_loss: 1.1776
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1977 - val_loss: 1.1621
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1454 - val_loss: 1.1464
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1675 - val_loss: 1.1302
Epoch 9/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1373 - val_loss: 1.1142
Epoch 10/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1409 - val_loss: 1.0985
Epoch 11/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0675 - val_loss: 1.0832
Epoch 12/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0

KeyboardInterrupt: 

In [10]:
# Apply the aggregate explainer on a sample instance:
sample_idx = 0
agg_explanation = agg_explainer.explain_instance(X_test.iloc[sample_idx])

# Display the aggregated explanation and the ranking dataframe:
print("Feature importance scores:")
display(agg_explanation)                    # Displays each feature and their importance scores
print("\nFeature importance ranking:")
display(get_ranked_explanation(agg_explanation))    # Displays the ranking of the features based on their importance scores; features with similar importance scores are grouped together

Feature importance scores:


,feature,score
0,Duration,1.060561
1,Age,0.883932
2,Checking_account_none,0.758400
3,Checking_account_little,0.682422
4,Purpose_furniture_equipment,0.670604
5,Checking_account_moderate,0.318891
6,Credit_amount,0.213071
7,Housing_free,0.177050
8,Saving_accounts_moderate,0.100845
9,Sex_female,0.096998



Feature ranking:


,Feature,Rank
0,Duration,1
1,Age,2
2,Checking_account_none,3
3,Checking_account_little,4
4,Purpose_furniture_equipment,4
5,Checking_account_moderate,5
6,Credit_amount,6
7,Housing_free,7
8,Saving_accounts_moderate,8
9,Sex_female,8


### Get information on the aggregate explainer's last explanation
With the `get_last_explanation_info()` method, you can get a dataframe that contains each of the aggregated explanation models' performances on each of the metrics used to evaluate them. You are also given the weight each explanation model got from the MCDM algorithm, which is passed on to the rank aggregation step.

In [9]:
agg_explainer.get_last_explanation_info()

,nrc,sensitivity_spearman,faithfulness_corr,weight
LimeWrapper,46.282592,0.870936,0.269322,0.335610
ShapTabularTreeWrapper,42.648201,0.967775,0.219899,0.431085
AnchorWrapper,16.854363,0.672946,0.569971,0.585786


# Evaluating the aggregate explainer

### The ExplanationModelEvaluator Class
This class holds all definitions for the metrics used to evaluate the explanation models. The aggregate explainer maintains an instance of this class in order to use its evaluations in the aggregation process. It is designed so that it can be used on any explainer that follows the interface and behavior conventions of the `explainers.py` file.

### Using the internal ExplanationModelEvaluator instance
In order to be used, the ExplanationModelEvaluator class must be instantiated and its `init()` method must be called. This process, however, is somewhat time-consuming, since one of the metrics defined by this class relies on generating a noisy variation of the training data, and, to do that, an autoencoder is trained with tensorflow.

However, this is usually not necessary, since the AggregateExplainer class maintains its own instance of the ExplanationModelEvaluator class, which can be used normally.

In [12]:
# ++ Usual instantiation of the ExplanationModelEvaluator class:
#
# evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
# evaluator.init()    # Takes some time to train the autoencoder

# ++ Or, grab the one maintained by the AggregatedExplainer:
evaluator = agg_explainer.xai_evaluator

### [WORKAROUND] Applying the sensitivity metric to the aggregate explainer:
One of the metrics defined in the ExplanationModelEvaluator class is the sensitivity metric. The way it works requires it to create several new instances of the explanation model being evaluated, since they each need to be fit to a different noisy variation of the training data. This process is very slow, and therefore multiprocessing is used in the `sensitivity()` function to distribute the workload. This, however, poses an issue when evaluating the sensitivity of the aggregate explainer model, since it may also use the sensitivity metric itself to perform the aggregation, which means a child process would have to create another child process, which usually is not allowed.

As of now, in order to apply the sensitivity metric to the aggregate explainer, you must use a variation of its implementation that does the calculation without multiprocessing. A sequential version of the `sensitivity()` metric is provided by the `_sensitivity_sequential()` function.

In [ ]:
evaluator._sensitivity_sequential(
    agg_explainer, 
    X_test.iloc[sample_idx],
    extra_explainer_params={    # Must specify everything the explainer needs to be instantiated
        "explainer_types": [LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper],
        "evaluator": agg_explainer.xai_evaluator # Remember to resue the same evaluator instance, otherwise the autoencoder will be retrained for every iteration
    },
    iterations=3,
)

0.9400656814449916

### Full evalution of the aggregate explainer

Here's one way of evaluating the aggregate explainer and comparing it to the explainers whose explanations were aggregated. In this example, the aggregate explainer was evaluated with the same metrics it used to internally evaluate each of the component models. The `get_last_explanation_info()` function was used to retrieve the metrics that were calculated internally, so they aren't calculated twice.

In [15]:
faithfulness = evaluator.faithfullness_correlation(agg_explainer, X_test.iloc[sample_idx])
sensitivity = evaluator._sensitivity_sequential( # sequential version of sensitivity must be used at this time
                                                agg_explainer, X_test.iloc[sample_idx],
                                                extra_explainer_params={
                                                    "explainer_types": [LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper],
                                                    "evaluator": agg_explainer.xai_evaluator
                                                },
                                                iterations=10
                                            )
nrc = evaluator.nrc(agg_explainer, X_test.iloc[sample_idx])

metrics = agg_explainer.get_last_explanation_info().drop(columns='weight')

metrics.at[AggregatedExplainer.__name__, 'faithfulness_corr'] = faithfulness
metrics.at[AggregatedExplainer.__name__, 'sensitivity_spearman'] = sensitivity
metrics.at[AggregatedExplainer.__name__, 'nrc'] = nrc

In [16]:
metrics

,nrc,sensitivity_spearman,faithfulness_corr
LimeWrapper,46.152620,0.859212,0.355147
ShapTabularTreeWrapper,42.648201,0.954843,0.154678
AnchorWrapper,18.442814,0.668667,0.079319
AggregatedExplainer,44.579487,0.913744,0.320685


#### Using the `utils.evaluate_aggregate_explainer()` function
Utility function to evaluate the aggregate explainer, varying its settings. For each of the aggregate explainer's parameters (explainer components, mcdm algorighm, aggregation algorithm), the function accepts a list of possible values; it'll iterate over every possible value combination, checking n_instances, and will return the results as a list of lists of dataframes, one dataframe for each instance check, and one list of dataframes for each setting configuration.

In [6]:
from xai_agg.utils import evaluate_aggregate_explainer

results = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,                                         # Model and data
    explainer_components_sets=[[LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper]],   # Wrapped explainer sets to be tested
    mcdm_algs=[pymcdm.methods.TOPSIS()],                                                # MCDM algorithms to be tested
    aggregation_algs=["wsum"],                                                          # Aggregation algorithms to be tested
    metrics_sets=[['nrc', 'sensitivity_spearman', 'faithfulness_corr']],                # Metric sets to be tested
    n_instances=1,                                                                      # Number of instances per setting to run the evaluation on
    mp_jobs=5                                                                           # Number of jobs to run in parallel (DECREASE THIS VALUE WHEN LOW RAM IS AVAILABLE)
)

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.2613 - val_loss: 1.2377
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2376 - val_loss: 1.2201
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2341 - val_loss: 1.2033
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1956 - val_loss: 1.1861
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2065 - val_loss: 1.1692
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1557 - val_loss: 1.1520
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1403 - val_loss: 1.1354
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1075 - val_loss: 1.1189
Epoch 9/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0877 - val_loss: 1.1028
Epoch 10/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1104 - val_loss: 1.0876
Epoch 11/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0776 - val_loss: 1.0724
Epoch 12/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.

In [8]:
results

[[                              nrc  sensitivity_spearman  faithfulness_corr
  LimeWrapper             43.895319              0.849754           0.561030
  ShapTabularTreeWrapper  49.558466              0.988480           0.074414
  AnchorWrapper           37.743216              0.784517           0.615187
  AggregateExplainer      44.208543              0.955355           0.720372]]

In [9]:
# Get mean results for a specific setting:

desired_setting = 0
mean_results = pd.concat(results[desired_setting]).groupby(level=0).mean()
display(mean_results)

,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,44.208543,0.955355,0.720372
AnchorWrapper,37.743216,0.784517,0.615187
LimeWrapper,43.895319,0.849754,0.561030
ShapTabularTreeWrapper,49.558466,0.988480,0.074414
